## LAB Assignment
Please finish the **Exercise** and answer **Questions**.
### Exercise 
In this lab, we will write a program to segment different objects in a video using *K-means* clustering. There are several steps:

-  1.Load video & extract frames
-  2.Implement *K-means* clustering
-  3.Write back to video


#### Import some libraries

In [1]:
import numpy as np
import cv2
import tqdm
import os
import sys
# color of different clusters
GBR = [[0, 0, 255],
       [0, 128, 255],
       [255, 0, 0],
       [128, 0, 128],
       [255, 0, 255]]

# path configuration
project_root = os.path.abspath('.')
output_path = os.path.join(project_root)
input_path = os.path.join(project_root)
if not os.path.exists(output_path):
    os.makedirs(output_path)
output_path

'e:\\AsciiStandardPath\\PracticeFile\\22fall\\P_Machine_Learning\\SUSTech-Machine-Learning-Lab-Solutions\\Solutions\\Lab11.K-means clustering'

#### implement K-means
In this lab, you need to implement K-means, the rough procedure is:

1. **initialize centroids** of different classes

   In the simplest case, randomly choose centroids in original data

2. **calculate distances** between samples (pixels) and centroids

   Since one sample (pixel) has 3 channels, you can calculate square sum of differences in each channel between it and centroids.
   $$
   dist(S,C) = \sum_{i=1}^3(C_i-S_i)^2 \\
   \left\{
   \begin{aligned}
   &dist(S,C): \text{distance between a sample S and a centroid C}\\
   &C: \text{a centroid}\\
   &S: \text{a sample}\\
   &S_i: \text{the } i^{th} \text{ channel's value of S}\\
   &C_i: \text{the } i^{th} \text{ channel's value of C}
   \end{aligned}
   \right.
   $$
   
3. **classify** every samples

   A sample is belonging to the class whose centroid is closest to it among all centroids.
   $$
   cls(S) = argmin(\sum_{i=1}^3(C_i^k-S_i)^2), k=1,2,...,K\\
   \left\{
   \begin{aligned}
   &cls(S): \text{class of a sample S}\\
   &K: \text{number of classes}\\
   &C^k: \text{centroid of } k^{th} \text{ class}\\
   \end{aligned}
   \right.
   $$

4. **update centroid**

   You can use mean of all samples in the same class to calculate new centroid.
   $$
   C^k_i =\frac{1}{n^k}\sum^{n^k}_{n=1}S^k_{in},\ \  i=1,2,3\\
   \left\{
   \begin{aligned}
   &C^k_i: \text{the } i^{th} \text{channel's value of a centroid belonging to the } k^{th} \text{class} \\
   &n^k: \text{the number of samples in the }  k^{th} \text{class}\\
   &S^k_{in}: \text{the } i^{th} \text{channel's value of a sample which is in the } k^{th} \text{class}
   \end{aligned}
   \right.
   $$
   
5. loop until classification result doesn't change



In addition, you may find there is code like this:

```python
while ret:
    frame = np.float32(frame)
    h, w, c = frame.shape
    ...
```

Since if you don't converse the `dtype`, K-means hardly converges which means it will stuck into dead loop easily.



After you finish K-means, you will find the written video is hard to watch because **color** between adjacent frames **changes almost all the time**. Here, I want you to find a way to alleviate the situation yourself.  有可能每一帧聚类颜色不一样。

**It isn't compulsory**, you can try if you want.






In [2]:
import torch as np
import random
np.array = np.Tensor
np.ndarray = np.TensorType
np.array.astype = np.array.to
import numpy as real_np

def choose_k(data: np.ndarray, n_cl: int, weights: np.ndarray = None, seeds=42, device='cuda'):
    """choose k samples randomly"""
    # random.seed(seeds)
    # if weights is None:
    #     weights = np.ones((len(data),)).to(device) / len(data)
    # choices = np.zeros((n_cl, data.shape[1])).to(device)
    # for i in range(n_cl):
    #     a = random.random()
    #     for j, w in enumerate(weights):
    #         a -= w
    #         if a <= 0:
    #             # choices = np.concatenate((choices, data[j].unsqueeze(0)), 0)
    #             choices[i] = data[j]
    #             break
    if weights is not None:
        weights = weights.cpu().numpy()
    choices = real_np.random.choice(data.shape[0], n_cl, p=weights)
    return data[choices]
a = np.arange(9).reshape(3,3).to('cuda')
choose_k(a, 2, device='cuda')


tensor([[3, 4, 5],
        [0, 1, 2]], device='cuda:0')

In [3]:
def nearest_distances(data, existing_centers):
    # result = np.zeros(len(data))
    # for i in range(len(data)):
    #     result[i] = np.linalg.norm(data[i] - existing_centers, axis=1).min()
    # return result
    return np.min(np.linalg.norm(data.unsqueeze(1) - existing_centers, axis=2), axis=1)[0]
a = a.astype(np.float64).to('cuda')
nearest_distances(a, a[0:2])

tensor([0.0000, 0.0000, 5.1962], device='cuda:0', dtype=torch.float64)

In [4]:
def arg_nearest_distances(data, existing_centers):
    # result = np.zeros(len(data))
    # for i in range(len(data)):
    #     result[i] = np.linalg.norm(data[i] - existing_centers, axis=1).argmin()
    # return result
    return np.linalg.norm(data.unsqueeze(1) - existing_centers, axis=2).argmin(axis=1) # 进一步优化
a = a.astype(np.float64).to('cuda')
arg_nearest_distances(a, a[0:2])

tensor([0, 1, 1], device='cuda:0')

In [5]:
def init_centers_kpp(data: np.ndarray, n_cl: int, seeds=42, device='cuda'):
    """kmeans++"""
    # centers = [data[random.randint(0, len(data) - 1)]]
    centers = np.array(n_cl, data.shape[1]).astype(np.float64).to(device)
    random.seed(seeds)
    centers[0] = data[random.randint(0, len(data) - 1)].astype(np.float64)
    # mean = centers[0]
    for i in range(1, n_cl):
        # 找每个点到已有中心的最近距离
        existing_centers = centers[:i]
        distances = nearest_distances(data, existing_centers)
        weights = distances / np.sum(np.array(distances))
        new_center = choose_k(data, 1, weights=weights, device=device).squeeze() # 越远的点被选中的概率越大
        centers[i] = new_center
        # mean += (new_center - mean) / (i + 1)
    return centers
a = np.arange(9).reshape(3,3).to('cuda')
init_centers_kpp(a, 3, device='cuda') 
init_centers_kpp(a, 2,device='cuda') 

tensor([[6., 7., 8.],
        [0., 1., 2.]], device='cuda:0', dtype=torch.float64)

In [6]:
import tqdm
from tqdm import trange
from concurrent.futures import ThreadPoolExecutor
def kmeans(data: np.ndarray, n_cl: int, maxiter=1000, seeds=42, device='cuda'):
    """
        K-means

    :param data:    original data
    :param n_cl:    number of classes
    :param seeds:   seeds
    :return:        new labels and new seeds
    """
    n_samples, channel = data.shape

    # firstly you should init centroids by a certain strategy
    centers = init_centers_kpp(data, n_cl, seeds=seeds, device=device)
    # centers = choose_k(data, n_cl, seeds=seeds, device=device)
    # centers = random.choices(data, k=n_cl)

    old_labels = np.zeros((n_samples,)).to(device)
    def spin_once():
        nonlocal old_labels
        nonlocal centers
        iters.set_description(f'kmeans iter {j}')
        # calc distance between samples and entroids
        # classify samples
        new_labels = arg_nearest_distances(data, centers) # 

        # update centroids
        centers = np.zeros((n_cl, channel)).to(device)
        for i in range(n_cl):
            centers[i] = np.mean(data[new_labels == i], axis=0)
        if np.all(new_labels == old_labels):            
            return False
        old_labels = new_labels
        return True
    iters = trange(maxiter)
    for j in iters:
        if not spin_once():
            break
    return old_labels, centers
kmeans(a.astype(np.float), 2, 1000, seeds=42, device='cuda')

kmeans iter 1:   0%|          | 1/1000 [00:00<00:04, 199.96it/s]


(tensor([1, 0, 0], device='cuda:0'),
 tensor([[4.5000, 5.5000, 6.5000],
         [0.0000, 1.0000, 2.0000]], device='cuda:0'))

#### Load video and detect
We use `opencv` to read a video.
<font color=red>Pay attention</font> that data type of `frame` is `uint8`, not `int`; In this lab, frame has 3 channels.
If you don't change `dtype` of frame into `unit8`, video you write will look strange which you can have a try.

In [7]:
import numpy as real_np
cu_GBR = np.array(GBR).to('cuda')

def process_frame(frame, n_cl, seeds, device, samples=400*400, prev_centers=None, location_factor = 0.05):
    random.seed(42)
    real_np.random.seed(42)
    frame = frame.astype(np.float64).to(device)
    h, w, c = frame.shape

    # k-means
    data = frame.reshape((h * w, c))
    
    # 引入位置信息，减少闪烁问题
    # location_factor = 0.01
    # location_factor = 0.05
    # location_factor = 0.1
    # location_factor = 0.3
    # location_factor = 10000
    
    location_info = np.zeros((h*w, 2)).to(device)
    location_info[:, 0] = np.arange(h*w) // w * location_factor
    location_info[:, 1] = np.arange(h*w) % w * location_factor
    
    data = np.hstack((data, location_info))
    
    # subset_data = choose_k(data, 40000, seeds=seeds, device=device)
    subset_indices = real_np.random.choice(len(data), samples, replace=False)
    subset_data = data[subset_indices]
    
    # location_info = np.zeros((samples, 2)).to(device)
    # location_info[:, 0] = subset_indices // w
    # location_info[:, 1] = subset_indices % w
    
    # train_data = np.hstack((subset_data, location_info))
   
            
    k_result = kmeans(subset_data, 
                      n_cl=n_cl, maxiter=200, seeds=42,device=device)
    # labels = k_result[0].astype(np.int)
    centers = k_result[1]
    labels = arg_nearest_distances(data, centers).astype(np.int)
    
    
    # # 为了防止颜色闪烁问题，我们做一个配准。
    # prev_lables = prev_centers
    # if prev_lables is not None:
    #     prev_hist = np.bincount(prev_lables.reshape(-1))
    #     hist = np.bincount(labels.reshape(-1))
    #     # 按照频率排序来配准
    #     prev_order = np.argsort(prev_hist) # 大小->类别
    #     order = np.argsort(hist) # 大小->类别
    #     inv_order = order[np.arange(len(order))] # 类别->大小
    #     map_curr2pre =  prev_order[inv_order]# 类别->类别
    #     lables = map_curr2pre[labels]
    # prev_centers = prev_lables
    
    
    # 效果不太好，我们再换一个方法。我们根据每个类别的中心点来配准。
    if prev_centers is not None:
        # 计算每个类别的中心点和上一帧的中心点的距离
        dist = np.zeros((n_cl, n_cl)).to(device)
        for i in range(n_cl):
            for j in range(n_cl):
                dist[i, j] = np.linalg.norm(centers[i]-prev_centers[j])
        # 按照距离来配准
        map_curr2pre = np.argmin(dist, axis=1)
        labels = map_curr2pre[labels]
    prev_centers = centers
    
    # give different cluster different colors
    # new_frame = np.zeros((h * w, c)).to(device)
    #  dye pixels with colors
    new_frame = cu_GBR[labels]

    new_frame = new_frame.reshape((h, w, c)).astype(np.uint8)
    return new_frame, prev_centers

device = np.device("cuda:0" if np.cuda.is_available() else "cpu")
n_cl = 3
# frame = np.rand((5,5,3)).astype(np.float64).to(device)
frame = np.rand((1200,1200,3)).astype(np.float64).to(device)
new_frame = process_frame(frame, n_cl, 42, device)
new_frame[0].shape

kmeans iter 72:  36%|███▌      | 72/200 [00:00<00:00, 385.02it/s]


torch.Size([1200, 1200, 3])

In [8]:
prev_hist = np.arange(9)
hist = np.array([8, 7, 6, 5, 4, 3, 2, 1, 0])
lables = np.arange(9).reshape(3,3)

prev_order = np.argsort(prev_hist) # 大小->类别
order = np.argsort(hist) # 大小->类别
inv_order = order[np.arange(len(order))] # 类别->大小
map_curr2pre =  prev_order[inv_order]# 类别->类别
lables = map_curr2pre[lables]
lables

tensor([[8, 7, 6],
        [5, 4, 3],
        [2, 1, 0]])

In [9]:
# h, w = 400, 400
# samples = h*w
# location_info = np.zeros((samples, 2)).to(device)
# for x in range(h):
#     for y in range(w):
#         location_info[x * w + y] = np.array([x, y]).to(device)
# location_info.shape

In [10]:
import numpy as real_np
device = np.device("cuda:0" if np.cuda.is_available() else "cpu")
def detect(video, n_cl=2, location_factor=0.01):
    # load video, get number of frames and get shape of frame
    cap = cv2.VideoCapture(video)
    fps = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    # instantiate a video writer
    path = os.path.join(output_path, f"result_with_{n_cl}clz-{location_factor}lf.mp4")
    if os.path.exists(path):
        return
    video_writer = cv2.VideoWriter(os.path.join(output_path, f"result_with_{n_cl}clz-{location_factor}lf.mp4"),
                                   cv2.VideoWriter_fourcc(*'mp4v'),
                                   (fps / 10),
                                   size,
                                   isColor=True)

    # initialize frame and seeds
    ret, frame = cap.read()
 

    # print("Begin clustering with %d classes:" % n_cl)
    # bar = tqdm.tqdm(total=fps)  # progress bar
    prev_centers = None
    while ret:
        # frame = np.float32(frame)
        frame = np.array(frame)
        new_frame, prev_centers = process_frame(frame, n_cl, 42, device, prev_centers=prev_centers, location_factor=location_factor)
        video_writer.write(new_frame.cpu().numpy())

        ret, frame = cap.read()
        # bar.update()

    # release resources
    video_writer.release()
    cap.release()
    cv2.destroyAllWindows()


video_sample = os.path.join(input_path, "road_video.MOV")
# detect(video_sample, n_cl=5)


In [14]:
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures as futures
tasks = []
with ThreadPoolExecutor(max_workers=32) as executor:
    for k in range(2, 6):
    # for k in range(3, 4):
    # for k in range(4, 5):
    # for k in range(5, 6):
        # for location_factor in [0, 0.01, 0.1, 100]:
        for location_factor in [0, 0.1, 100]:
            tasks.append(executor.submit(detect, video_sample, n_cl=k, location_factor=location_factor))
            
    for task in tqdm.tqdm(futures.as_completed(tasks), total=len(tasks)):
        pass
    

  0%|          | 0/3 [00:00<?, ?it/s]


















































kmeans iter 48:  24%|██▍       | 48/200 [00:00<00:01, 109.84it/s]












kmeans iter 57:  28%|██▊       | 57/200 [00:00<00:01, 76.92it/s] 





















kmeans iter 29:  14%|█▍        | 29/200 [00:00<00:02, 66.49it/s]









































































kmeans iter 86:  43%|████▎     | 86/200 [00:00<00:01, 98.62it/s]



























kmeans iter 90:  45%|████▌     | 90/200 [00:00<00:01, 101.62it/s]



































































kmeans iter 86:  43%|████▎     | 86/200 [00:00<00:01, 97.33it/s]




























kmeans iter 86:  43%|████▎     | 86/200 [00:00<00:01, 96.56it/s]












































kmeans iter 26:  13%|█▎        | 26/200 [00:00<00:01, 111.29it/s]























kmeans iter 86:  43%|████▎     | 86/200 [00:01<00:01, 79.44it/s]




















kmeans i

我们刚才把所有的location_factor与k的组合都试了一遍，生成了所有的视频。

可以观察到，location_factor对于防闪烁有一定的作用，但是如果设置太大，可能会导致全屏只按照位置而不是视频内容去聚类。

对于k我们可以观察到，k越大，聚类的效果越好，视频的内容更加精细。

### Questions

1. What are the strengths of K-means; when does it perform well?

优点：
- 简单
    - 容易理解
- 快速
    - 容易并行。
    - 可以用GPU加速。
    - 可以用多线程加速


2. What are the weaknesses of K-means; when does it perform poorly?
- 需要预先指定K

- 数据量大的时候，收敛很慢。这个可以用mini-batch Kmeans解决，就是采样。

- 对于非凸数据集，可能会收敛到局部最优解

- 对于流形学习那种复杂的结构，Kmeans学不出那个结构，因为欧几里得距离不太使用。

3. What makes K-means a good candidate for the clustering problem, if you have enough knowledge about the data?

当
- 数据噪声较少，数据量不大
- 数据距离用欧几里得距离度量比较合理
- 数据近似服从高斯混合分布
时，Kmeans是一个不错的选择。
